![Trie](../static/trie.png)

^example of what a **TRIE** may look like

- notice how root is empty
- all descendants of a node have a common prefix
- widely used in autocomplete, spell checkers
- can represent a trie with an array or a hashmap (also data structures)

This is how you insert into trie

![insert](../static/insert-trie.gif)

and how you search in a Trie

![search](../static/search-trie.gif)

**Trie** is also sometimes called **prefix tree**, it is a special form **N-ary tree** (meaning each node can have no more than **N** children)

Origin of **Trie** is from re**trie**ve

You use an array or a hashmap to implement a **Trie**. For example

In [21]:
from typing import Tuple
import json


class TrieNode:
    def __init__(self, char: str):
        self.char = char
        self.children = []
        self.word_finished = False  # Is it the last character of the word ?
        self.counter = 1            # How many times this character appeared in the addition process

    # could have used __str__ as well. __repr__ conventional use is to reconstruct
    # the instance of a class given a string. For example, if class Foo has a single
    # attribute bar, and its value is 'bar', then __repr__ would return
    # Foo(bar='bar'). Here we use __repr__ to help us visualize the Trie
    def __repr__(self):
        return json.dumps(self._json(), indent=4)

    def _json(self):
        return {
            "char": self.char,
            "children": [child._json() for child in self.children],
            "word_finished": self.word_finished,
            "counter": self.counter,
        }

# adds a word to a TrieNode
#
# 1. current node is root at start
# 2. for each character in the word
#    - look for the TrieNode whose value (self.char) == this character
#    - if can't find, create a TrieNode; else traverse down the found TrieNode
#
# Time Complexity of this is O(N * M). Meaning very inefficient
# N - length of word
# M - maximum number of children a TrieNode can have (a Trie is a special form N-ary Trie)
#
# How could we improve the time complexity of this algo?
# We could store pointers to the locations of the nodes separately. For example, given the root (its
# hash, or some sort of key), we can look up the pointer to it in a database that would hold this info
# for us. We can now get the instance of the root TrieNode, what next? We can now use the first character
# of the word as an index, to go to the db to get the pointer to the next TrieNode, and so on.
# You will agree that this is much more efficient. We have paid the price of extra storage in return for
# the greater performance (time complexity)
# Our TrieNode now, would at most take
# Time Complexity O(N)
# with Space Complexity O(L), where L is the total number of TrieNodes (many of TrieNode)
# If you look here: https://eth.wiki/en/fundamentals/patricia-tree
# then that is exactly how it is done in Ethereum. The only caveat, Ethereum doesn't use Tries, at least,
# not the basic prefix tries. Let's continue exploring to understand why
def add(root, word: str):
    node = root
    for char in word:
        found_in_child = False
        for child in node.children:
            if child.char == char:
                child.counter += 1
                node = child
                found_in_child = True
                break

        if not found_in_child:
            new_node = TrieNode(char)
            node.children.append(new_node)
            node = new_node

    node.word_finished = True


# def find_prefix(root, prefix: str) -> Tuple[bool, int]:
#     node = root
#     if not root.children:
#         return False, 0
#     for char in prefix:
#         char_not_found = True
#         for child in node.children:
#             if child.char == char:
#                 char_not_found = False
#                 node = child
#                 break
#         if char_not_found:
#             return False, 0
#     return True, node.counter


# print(find_prefix(root, 'hac'))
# print(find_prefix(root, 'hack'))
# print(find_prefix(root, 'hackathon'))
# print(find_prefix(root, 'ha'))
# print(find_prefix(root, 'hammer'))

In [36]:
root = TrieNode('*')
add(root, "dog")
add(root, "done")
add(root, "dope")

In [37]:
root

{
    "char": "*",
    "children": [
        {
            "char": "d",
            "children": [
                {
                    "char": "o",
                    "children": [
                        {
                            "char": "g",
                            "children": [],
                            "word_finished": true,
                            "counter": 1
                        },
                        {
                            "char": "n",
                            "children": [
                                {
                                    "char": "e",
                                    "children": [],
                                    "word_finished": true,
                                    "counter": 1
                                }
                            ],
                            "word_finished": false,
                            "counter": 1
                        },
                        {
                        

### REITERATION. WHY DON'T WE USE TRIE IN ETHEREUM?

- awful time complexity
- even if improved like in the above, wastes space (see radix trie below for comparison)

### INTRODUCING RADIX TRIE

![trie-vs-radix](../static/trie-vs-radix.png)

![trie-vs-radix-2](../static/trie-vs-radix-2.png)

i.e. the RADIX TRIE uses the space better. So RADIX TRIE is just like TRIE, but with better space

Here is Radix Tree

Merkle / PATRICIA (Practical Algorithm To Retrieve Information Coded in Alphanumeric)

Trie

Modified Merkle Trie is Ethereum's optimized Merkle Trie

**Merkle Patricia Trie** (or Modified Merkle Patricia Trie, same thing) - is a data structure in computer science. In other words, it is a model to store data. You can learn about the Trie data structure here: https://www.wikiwand.com/en/Trie#/:~:text=In%20computer%20science,%20a%20trie,the%20keys%20are%20usually%20strings.

**MPT** is:

- CRYPTOGRAPHICALLY AUTHENTICATED

- can store all (key, value) bindings

- O(log N) INSERT, LOOKUP, DELETE